In [98]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

import boston_valuation_tool as val

In [2]:
# Gather data
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'],  axis=1)

log_prices=np.log(boston_dataset.target)
target = pd.DataFrame(data=log_prices, columns=['PRICE'])
np.shape(features)

(506, 11)

In [73]:
CRIME_IDX = 0
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

property_stats = np.ndarray(shape=(1, 11))
property_stats = features.mean().values.reshape(1, 11)
features

CRIM    ZN  CHAS    NOX     RM     DIS  RAD    TAX  PTRATIO       B  \
0    0.00632  18.0   0.0  0.538  6.575  4.0900  1.0  296.0     15.3  396.90   
1    0.02731   0.0   0.0  0.469  6.421  4.9671  2.0  242.0     17.8  396.90   
2    0.02729   0.0   0.0  0.469  7.185  4.9671  2.0  242.0     17.8  392.83   
3    0.03237   0.0   0.0  0.458  6.998  6.0622  3.0  222.0     18.7  394.63   
4    0.06905   0.0   0.0  0.458  7.147  6.0622  3.0  222.0     18.7  396.90   
..       ...   ...   ...    ...    ...     ...  ...    ...      ...     ...   
501  0.06263   0.0   0.0  0.573  6.593  2.4786  1.0  273.0     21.0  391.99   
502  0.04527   0.0   0.0  0.573  6.120  2.2875  1.0  273.0     21.0  396.90   
503  0.06076   0.0   0.0  0.573  6.976  2.1675  1.0  273.0     21.0  396.90   
504  0.10959   0.0   0.0  0.573  6.794  2.3889  1.0  273.0     21.0  393.45   
505  0.04741   0.0   0.0  0.573  6.030  2.5050  1.0  273.0     21.0  396.90   

     LSTAT  
0     4.98  
1     9.14  
2     4.03  
3     2.94  
4     5.33  
..     ...  
501   9.67  
502   9.08  
503   5.64  
504   6.48  
505   7.88  

[506 rows x 11 columns]

In [46]:
regr = LinearRegression().fit(features, target)
fitted_values = regr.predict(features)
mse = mean_squared_error(fitted_values, target)
rmse = np.sqrt(mse)

In [47]:
def get_log_estimate(nr_room,
                    student_per_classroom,
                    next_to_river=True,
                    high_confident=False):
    
    # Configure Property
    property_stats[0][RM_IDX] = nr_room
    property_stats[0][PTRATIO_IDX] = student_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0
     
    # Make prediction
    log_estimate = regr.predict(property_stats)[0][0]
    
    # Calc Range
    if high_confident:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
    
    return log_estimate, upper_bound, lower_bound, interval

In [48]:
get_log_estimate(3, 20, next_to_river=True)

(2.776758191480399, 2.9642703266775294, 2.589246056283269, 68)

In [61]:
ZILLOW_MEDIAN_PRICE = 583.3
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)

log_est, upper, under, conf = get_log_estimate(9, student_per_classroom=15,
                                              next_to_river=False, high_confident=False)

# Convert to today dollars
dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
dollar_low = np.e**under * 1000 * SCALE_FACTOR

# Round to nearest thounsand
rounded_est = round(dollar_est, -3)
rounded_hi = round(dollar_hi, -3)
rounded_low = round(dollar_low, -3)

print(f'The estimated property value is {rounded_est}')
print(f'At {conf}% confidence the valuation range is')
print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

The estimated property value is 827000.0
At 68% confidence the valuation range is
USD 685000.0 at the lower end to USD 997000.0 at the high end.


In [72]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    """Estimate the price of a property in Boston
    
    Keyword arguments:
    rm -- number of rooms in the property
    ptratio -- number of students per teacher in the classroom for school in the area
    chas -- True if  the property is next to the river, False otherwise
    large_range -- True for a 95% prediction interval, False for a 68% interval
    
    """
    
    if rm < 1 or ptratio < 1:
        print('That is unrealistic, try again')
        return
    
    log_est, upper, under, conf = get_log_estimate(rm, student_per_classroom=ptratio,
                                                  next_to_river=chas, high_confident=large_range)

    # Convert to today dollars
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**under * 1000 * SCALE_FACTOR

    # Round to nearest thounsand
    rounded_est = round(dollar_est, -3)
    rounded_hi = round(dollar_hi, -3)
    rounded_low = round(dollar_low, -3)

    print(f'The estimated property value is {rounded_est}')
    print(f'At {conf}% confidence the valuation range is')
    print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

In [70]:
get_dollar_estimate(rm=6, ptratio=10, chas=True)

The estimated property value is 844000.0
At 95% confidence the valuation range is
USD 580000.0 at the lower end to USD 1228000.0 at the high end.


The estimated property value is 783000.0.
At 95% confidence the valuation range is
USD 538000.0 at the lower end to USD 1139000.0 at the high end.


In [100]:
val.get_dollar_estimate(6, 12, True)

The estimated property value is 783000.0.
At 95% confidence the valuation range is
USD 538000.0 at the lower end to USD 1139000.0 at the high end.
